In [78]:
import pandas as pd
from skimage import color
from skimage.io import imread
from skimage.feature import greycomatrix, greycoprops
import numpy as np

In [127]:
df = pd.read_csv("./local_path.csv")

In [128]:
props = ['contrast','dissimilarity','homogeneity','ASM','energy','correlation']

In [129]:
for i in range(len(df)):
    s = df.local_path.loc[i]
    if "DS_Store" in s:
        df = df.drop(i)
        
df.reset_index(drop=True,inplace=True)

In [130]:
len(df)

6927

In [131]:
brightness = []
n_channels = []
vector_list = []

for file in df.local_path:    
    img = imread(file)
    channels = len(img.shape)
    n_channels.append(channels)
    img_gray = color.rgb2gray(img)
    
    # because the bw are [0,255] and the color are [0,1]
    if channels == 2:
        val_mean = np.mean(img_gray) / 255    
    else:
        val_mean = np.mean(img_gray)
        
    brightness.append(val_mean)
    
    glcm = greycomatrix(img_gray, [1], [0], levels=256, symmetric=True, normed=True)
    
    prop_vector = []
    
    for prop in props:
        tmp = greycoprops(glcm, prop)
        prop_vector.append(float(tmp))
        
    vector_list.append(np.asarray(prop_vector + [val_mean]))
    
X = np.vstack(vector_list)
    
df['val'] = brightness
df['n_channels'] = n_channels

In [132]:
i_strings = []
for i in range(len(df)):
    s = df.local_path.loc[i]
    i_string = s[45:].split("/")[0]
    i_strings.append(i_string)

In [133]:
df['i_strings'] = i_strings

In [134]:
from PIL import Image, ImageDraw
from shapely.geometry import Point
import os
from sklearn.cluster import KMeans

In [135]:
k = 64

kmeans = KMeans(n_clusters = k)
kmeans.fit(X)

#df = df[:len(vector_list)]

df['clusters'] = kmeans.labels_

In [136]:
len(df)

6927

In [137]:
import math

In [138]:
side = int(math.ceil( np.sqrt(len(df)) ))

In [139]:
thumb_side = 64

In [140]:
px_w = side * thumb_side
px_h = side * thumb_side

In [141]:
df.sort(["clusters","val"],ascending=False,inplace=True)

In [142]:
df.reset_index(drop=True,inplace=True)

In [143]:
canvas = Image.new('RGB',(px_w, px_h),(50,50,50))
x,y = range(side) * side, np.repeat(range(side),side)
df['x'] = x[:len(df)]
df['y'] = y[:len(df)]

In [144]:
len(df)

6927

In [126]:
for i in range(len(df)):
    im = Image.open(df.local_path.loc[i])
    im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
    x_coord = df['x'].loc[i] * thumb_side
    y_coord = df['y'].loc[i] * thumb_side
    
    pos_i = (40,5)
    pos_i_string = (5,50)
    i_string = df.i_strings.loc[i]
    
    draw = ImageDraw.Draw(im)
    
    if i < 5600:
        draw.text(pos_i, str(i),fill="black")
        draw.text(pos_i_string,i_string,fill="black")
    else:
        draw.text(pos_i, str(i),fill="white")
        draw.text(pos_i_string,i_string,fill="white")
    
    canvas.paste(im,(x_coord,y_coord))
    
canvas.save("./white_pages_after.png")

In [96]:
# dangerous move here, but deleting white pages...and more doubles

In [97]:
import os

In [98]:
white_pgs = [56,60,62,64,66,67,71,72,73,75,449,451,452,454,456,457,458,460,461,462,465,466,468,469,470,474,477,485,487,710,711,712,713,714,715,716,717,718,719,721,722,725,726,731,734,1006,1069,1070,1073,1074,1076,1079,1081,1084,1499,1500,1501,1502,1504,1505,1509,1510,1514,1521,1623,1624,1625,1626,1627,1628,1629,1630,1631,1858,1861,1862,1865,1867,2106,2109,2110,2111,2115,2350,2395,2402,2742,2743,2745,2746,2747,2751,3030,3036,3143,3142,3141,3140,3139,3147,3148,3150,3151,3152,3158,3565,3566,3593,3597,3598,3599,3607,3616,3618,3747,3753,3756,3781,3986,3989,3995,3996,4272,4273,4274,4276,4277,4278,4279,4280,4282,4283,4287,4291,4292,4293,4294,4297,4298,4301,4642,4648,4764,4766,4767,4768,4769,4770,4772,4773,4774,4776,4777,4778,4779,4931,4932,4933,4936,5201,5202,5203,5206,5207,5214,5224,5228,5258,5259,5264,5427,5806,6372,6387,6466,6482,6484,6485,6489,6491,6492,6493,6497,6498,6499,6508,6510,6511,6687,6742,6747,6748,6749,6752,6756,6761,6905]

In [99]:
len(df)

7124

In [100]:
for i in white_pgs:
    os.remove(df.local_path[i])

In [101]:
df = df.drop(white_pgs)

In [102]:
len(df)

6927

In [103]:
df.reset_index(drop=True,inplace=True)

In [104]:
for i in range(len(df)):
    if i==0:
        val = df.val.loc[i]
    else:
        tmp = df.val.loc[i]
        if tmp==val:
            os.remove(df.local_path.loc[i])
        else:
            val = df.val.loc[i]

In [105]:
len(df)

6927

In [106]:
df.to_csv("./local_path.csv",index=False)

In [44]:
# now run create_path_table.ipynb...

In [107]:
df = pd.read_csv("./local_path.csv")

In [108]:
len(df)

7059